# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [13]:
city_data = "output_data/city_data.csv"
city_data
city_data = pd.read_csv("city_data.csv")
city_data.head()
city_data.dropna(inplace = True) 
city_data.head()
city_data.rename(columns={'Wind Speed': 'wind_speed'}, inplace=True)
city_data.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,lat,Lng,max_temp,wind_speed
0,0,albany,40.0,US,1.587837e+09,25.0,42.60,-73.97,64.00,4.70
1,1,vanimo,38.0,PG,1.587837e+09,83.0,-2.67,141.30,79.65,5.75
2,2,iqaluit,90.0,CA,1.587837e+09,92.0,63.75,-68.51,32.00,11.41
3,3,port alfred,31.0,ZA,1.587837e+09,91.0,-33.59,26.89,61.00,4.00
4,4,belushya guba,63.0,RU,1.587838e+09,51.0,59.70,30.79,37.99,11.18


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:
# Store latitude and longitude in locations
locations = city_data[["lat", "Lng"]]

# Fill NaN values and convert to float
humidity = city_data["Humidity"].astype(float)


In [15]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
city_data.dropna(how='all', inplace=True) 
city_data.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,lat,Lng,max_temp,wind_speed
0,0,albany,40.0,US,1.587837e+09,25.0,42.60,-73.97,64.00,4.70
1,1,vanimo,38.0,PG,1.587837e+09,83.0,-2.67,141.30,79.65,5.75
2,2,iqaluit,90.0,CA,1.587837e+09,92.0,63.75,-68.51,32.00,11.41
3,3,port alfred,31.0,ZA,1.587837e+09,91.0,-33.59,26.89,61.00,4.00
4,4,belushya guba,63.0,RU,1.587838e+09,51.0,59.70,30.79,37.99,11.18


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [36]:
#A max temperature lower than 80 degrees but higher than 70.
#temp_criteria = (city_data.max_temp <80) & (city_data.max_temp >70)
#Wind speed less than 10 mph.
#wind_criteria = (city_data.wind_speed <10)
#Zero cloudiness.
#cloud_criteria =(city_data.Cloudiness == 0)
#combine criteria
#final_criteria = temp_criteria & wind_criteria & cloud_criteria
#final_criteria
ideal_weather_df = city_data.loc[(city_data["max_temp"] > 70) & (city_data["max_temp"] < 80) & 
                                 (city_data["wind_speed"] < 10) & (city_data["Cloudiness"] == 0), :]
ideal_weather_df.dropna(how='all')
ideal_weather_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,lat,Lng,max_temp,wind_speed
116,116,manokwari,0.0,MX,1.587838e+09,44.0,30.48,-115.95,76.33,6.62
321,321,rawson,0.0,CL,1.587838e+09,99.0,-32.83,-70.60,72.00,1.01
401,401,yar-sale,0.0,IR,1.587838e+09,14.0,31.27,56.81,77.58,5.32
467,467,willowmore,0.0,MU,1.587838e+09,89.0,-19.98,57.61,75.99,4.00
501,501,sinkat,0.0,MG,1.587838e+09,78.0,-20.28,44.28,78.89,7.54


In [38]:
# Create a hotel_df
# The function below kept getting an error that it is no longer used in pandas
#hotel_df = ideal_weather_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
# I tried adding the column like below and it did not work... but why?
#hotel_df["Hotel Name"] = ""

hotel_df =  pd.DataFrame(ideal_weather_df, columns=["City", "Country", "Lat", "Lng"])
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
116,manokwari,MX,NaN,-115.95,
321,rawson,CL,NaN,-70.60,
401,yar-sale,IR,NaN,56.81,
467,willowmore,MU,NaN,57.61,
501,sinkat,MG,NaN,44.28,


In [42]:
hotels = []

for hotels in range(len(ideal_weather_df)):
    lat = ideal_weather_df.loc[hotels]['lat']
    lng = ideal_weather_df.loc[hotels]['Lng']

#    params = {
 #       "location": f"{lat},{lng}",
  #      "radius": 5000,
   #     "types" : "hotel",
    #    "key": g_key}
    
    #base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    #requested = requests.get(base_url, params=params)
    #json = requested.json()
    #try:
     #   hotels.append(json['results'][0]['name'])
    #except:
   #     hotels.append("")
#ideal_weather_df["Hotel Name"] = hotels
#ideal_weather_df = ideal_weather_df.dropna(how='any')
#ideal_weather_df.head()


KeyError: 0

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map